In [ ]:
%cp -R /kaggle/input/darknet-run-environment/darknet /kaggle/working


In [ ]:
%cp '/kaggle/input/predconf/2000.weights' '/kaggle/working/darknet'
%cp '/kaggle/input/predconf/run.py' '/kaggle/working/darknet'
%cp '/kaggle/input/predconf/obj.data' '/kaggle/working/darknet/data'
%cp '/kaggle/input/predconf/yolov4-custom.cfg' '/kaggle/working/darknet/cfg'
%cp '/kaggle/input/predconf/obj.names' '/kaggle/working/darknet/data'

In [ ]:
%cd /kaggle/working/darknet

!cp '../../input/libcuda/libcuda.so' .

!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!sed -i "s/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= ${ARCH_VALUE}/g" Makefile

!sed -i 's/LDFLAGS+= -L\/usr\/local\/cuda\/lib64 -lcuda -lcudart -lcublas -lcurand/LDFLAGS+= -L\/usr\/local\/cuda\/lib64 -lcudart -lcublas -lcurand -L\/kaggle\/working\/darknet -lcuda/' Makefile
!make &> compile.log

In [ ]:
!tail compile.log

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# from matplotlib.pyplot import figure
# import darknet
# import cv2

# network, class_names, class_colors = darknet.load_network(
#         'cfg/yolov4-custom.cfg',
#         'data/obj.data',
#         '2000.weights',
#         1
#     )

# ip_image = cv2.imread('/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/9674.jpg')
# darknet_image = darknet.make_image(640, 640, 3)
# ip_rgb = cv2.cvtColor(ip_image, cv2.COLOR_BGR2RGB)
# img_resized = cv2.resize(ip_rgb,(640,360) ,cv2.INTER_AREA)
# img_resized[360:640, 0:640] = (255,255,255)
# darknet.copy_image_from_bytes(darknet_image, img_resized.tobytes())
# detections = darknet.detect_image(network, class_names, darknet_image, 0.5)
# darknet.free_image(darknet_image)
# image = darknet.draw_boxes(detections, img_resized, class_colors)
# figure(figsize=(12.8, 12.8))
# plt.imshow(image)
# plt.show()
# preds = []
# for label, confidence, bbox in detections:
#     x1, y1, w1, h1 = bbox
#     if y1 < 360:
#         xmin, ymin, xmax, ymax = darknet.bbox2points(bbox)
#         preds.append(f'0.{confidence[0]} {int(xmin*2)} {int(ymin*2)} {int(w1*2)} {int(h1*2)}')
# prediction_str = ' '.join(preds)
# print(prediction_str)
# #sample_prediction_df['annotations'] = prediction_str
# #env.predict(sample_prediction_df)   # register your predictions

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import darknet
import cv2

network, class_names, class_colors = darknet.load_network(
        'cfg/yolov4-custom.cfg',
        'data/obj.data',
        '2000.weights',
        1
    )


for (pixel_array, sample_prediction_df) in iter_test:
    ip_image = pixel_array[:,:,::-1]   #cv2.imread('/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/9674.jpg')
    darknet_image = darknet.make_image(640, 640, 3)
    ip_rgb = cv2.cvtColor(ip_image, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(ip_rgb,(640,360) ,cv2.INTER_AREA)
    img_resized[360:640, 0:640] = (0,0,0)
    darknet.copy_image_from_bytes(darknet_image, img_resized.tobytes())
    detections = darknet.detect_image(network, class_names, darknet_image, 0.35)
    darknet.free_image(darknet_image)
    preds = []
    for label, confidence, bbox in detections:
        x1, y1, w1, h1 = bbox
        if y1 < 360:    
            xmin, ymin, xmax, ymax = darknet.bbox2points(bbox)
            preds.append(f'0.{confidence[0]} {int(xmin*2)} {int(ymin*2)} {int(w1*2)} {int(h1*2)}')
    prediction_str = ' '.join(preds)
    print(prediction_str)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)   # register your predictions
#     ifinal = darknet.draw_boxes(detections, img_resized, class_colors)
#     figure(figsize=(12.8, 12.8))
#     plt.imshow(ifinal)
#     plt.show()
#     cv2.imwrite('/kaggle/working/a.jpg', ifinal)

In [ ]:
import pandas as pd
sub_df = pd.read_csv('submission.csv')
sub_df.head()

In [ ]:
!cp /kaggle/working/darknet/submission.csv /kaggle/working
!rm -r /kaggle/working/darknet

In [ ]:
#import greatbarrierreef
#env = greatbarrierreef.make_env()# initialize the environment
#iter_test = env.iter_test()
#import pandas as pd
#df = pd.DataFrame()
#env.predict(df)